In [1]:
import pandas as pd
import string
import nltk
import os
import pickle
import re
from nltk.tokenize import RegexpTokenizer
import sklearn.feature_extraction.text as skft
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics as skmetrics
import sklearn.pipeline as skpipe
import sklearn.decomposition as skd
import sklearn.naive_bayes as sknb
from nltk.corpus import stopwords
import wordcloud
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score


C:\Users\vjk20\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
C:\Users\vjk20\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in 

Load dataset and create separate dataframe for phishing and non phishing emails

In [2]:
def create_phish_df(my_dir): #dataframe for phishing emails with columns as title.content and label as 1(phish)
    titles = []
    contents = []
    labels = []

    for f in os.listdir(os.path.join('',my_dir)):
            with open(os.path.join('', my_dir, f), 'r') as reader:
                try:
                    c = reader.read()
                except:
                    continue
                contents.append(c)
                titles.append(f)
                labels.append(1)

    df = pd.DataFrame({'title': titles, 'content': contents, 'label': 1},
                        columns = ['label', 'title', 'content'])
    return df
   

In [4]:
phish_email_list = [r"../Phishing-Detection/phish/20051114", 
r"../Phishing-Detection/phish/phishing0", 
r"../Phishing-Detection/phish/phishing1", 
r"../Phishing-Detection/phish/phishing2", 
r"../Phishing-Detection/phish/phishing3"]


In [5]:
phish_lst = []
for phish_folder in phish_email_list:
    phish_lst.append(create_phish_df(phish_folder))


In [6]:
df_phish = pd.concat(phish_lst)
df_phish = df_phish[:5000]


In [8]:
ham_email_list = [r"../Phishing-Detection/enron3", r"../Phishing-Detection/enron4", r"../Phishing-Detection/enron5",
 r"../Phishing-Detection/enron6"]


In [10]:
def create_ham_df(my_dir): #create dataframe for non phishing email set(ham means non phish) and label as 0.
                                      #The other columns are content and title
    titles = []
    contents = []
    labels = []

    for f in os.listdir(os.path.join(my_dir,'ham')):
            with open(os.path.join(my_dir, 'ham', f), 'r') as reader:
                try:
                    c = reader.read()
                except:
                    continue
                contents.append(c)
                titles.append(f)
                labels.append('0')

    df = pd.DataFrame({'title': titles, 'content': contents, 'label': 0},
                        columns = ['label', 'title', 'content'])
    return df

In [11]:
ham_list = []
for ham in ham_email_list:
    ham_list.append(create_ham_df(ham))


In [12]:
df_ham = pd.concat(ham_list)
df_ham = df_ham[:5000]
df_ham.head()

,label,title,content
0,0,0001.2001-02-07.kitchen.ham.txt,Subject: key hr issues going forward\na ) year...
1,0,0003.2001-02-08.kitchen.ham.txt,Subject: re : key hr issues going forward\nall...
2,0,0005.2001-02-08.kitchen.ham.txt,Subject: epmi files protest of entergy transco...
3,0,0006.2001-02-08.kitchen.ham.txt,Subject: california power 2 / 8\nplease contac...
4,0,0007.2001-02-09.kitchen.ham.txt,Subject: california power 2 / 9\nthe following...


In [13]:
df_emails = pd.concat([df_ham, df_phish])


In [14]:
df_emails_train, df_emails_test = train_test_split(df_emails, test_size=0.3, random_state=0)

 First text data is cleaned, by tokenising and lemmatising the text using wordnet, removing stop words,removing non-alphabetic strings. Bag-of-words approach (BOW) is used.  We look at the histogram of the words within the text, i.e. considering each word count as a feature.The intuition is that documents are similar if they have similar content. Further, that from the content alone we can learn something about the meaning of the document.

In [15]:
#frequency distribution
nltk.download('stopwords')
text_all = '\n'.join(df_emails_train.content).lower()
stop_words = set(stopwords.words('english')) #remove stopwords like a,an,for,the etc from the text
tokenizer = RegexpTokenizer(r'\w+')#nltk.tokenize.wordpunct_tokenize(text_all)
tokens_all = tokenizer.tokenize(text_all)
tokens_all = [word for word in tokens_all if word not in stop_words and word != 'font' and word != 'subject']#word not in string.punctuation

fd = nltk.probability.FreqDist(tokens_all)

phish_text_all = '\n'.join(df_phish.content).lower()
phish_tokens_all = tokenizer.tokenize(phish_text_all)
phish_tokens_all = [word for word in phish_tokens_all if word not in stop_words and word != 'font' and word != 'subject']

fd_phish = nltk.probability.FreqDist(phish_tokens_all)

pipeline = skpipe.Pipeline(
    steps = [('vect', skft.CountVectorizer(max_df=0.7)), #convert to numerical feature vectors
    ('tfidf', skft.TfidfTransformer()), #term frequency ,inverse document frequency
    ('clf', sknb.MultinomialNB())]) #multinomial naive bayes classification

df_emails_train, df_emails_test = train_test_split(df_emails, test_size=0.3, random_state=0)
pipeline.fit(df_emails_train.content, df_emails_train.label)

nb_test_predicted = pipeline.predict(df_emails_test.content)

titles = []
contents = []
labels = []

for f in os.listdir(os.path.join(r"../Phishing-Detection/enron5",'spam')):
        with open(os.path.join(r"../Phishing-Detection/enron5", 'spam', f), 'r') as reader:
            try:
                c = reader.read()
            except:
                continue
            contents.append(c)
            titles.append(f)
            labels.append(0)

df_spam = pd.DataFrame({'title': titles, 'content': contents, 'label': 0},
                    columns = ['label', 'title', 'content'])

predictions = pipeline.predict(df_spam.content)

df_spam['predicted_label'] = predictions

print(df_spam[0:5])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vjk20\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
   label                               title  \
0      0  0002.2001-05-25.SA_and_HP.spam.txt   
1      0  0004.2001-06-12.SA_and_HP.spam.txt   
2      0  0005.2001-06-23.SA_and_HP.spam.txt   
3      0  0006.2001-06-25.SA_and_HP.spam.txt   
4      0  0008.2001-06-25.SA_and_HP.spam.txt   

                                             content  predicted_label  
0  Subject: fw : this is the solution i mentioned...                1  
1  Subject: spend too much on your phone bill ? 2...                0  
2  Subject: discounted mortgage broker 512517\nmo...                0  
3  Subject: looking 4 real fun 211075433222\ntalk...                0  
4  Subject: your membership exchange , issue # 42...                0  


In [18]:
print(predictions)

[1 0 0 ... 0 0 0]


In [19]:
x= list(df_spam.label)
y= list(predictions)
print(len(x))
print(len(y))
results = confusion_matrix(x,y) 
#print('Confusion Matrix :')
print(results) 
print('Accuracy Score :',accuracy_score(x,y))
#print ('Report : ')
print(classification_report(x,y))

3675
3675
[[2700  975]
 [   0    0]]
Accuracy Score : 0.7346938775510204
              precision    recall  f1-score   support

           0       1.00      0.73      0.85      3675
           1       0.00      0.00      0.00         0

    accuracy                           0.73      3675
   macro avg       0.50      0.37      0.42      3675
weighted avg       1.00      0.73      0.85      3675

C:\Users\vjk20\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\classification.py:1438: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  recall = _prf_divide(tp_sum, true_sum,


In [21]:
predictions1=list(nb_test_predicted)
test_label=list(df_emails_test.label)
results = confusion_matrix(test_label, predictions1) 
#print('Confusion Matrix :')
print(results) 
print('Accuracy Score :',accuracy_score(test_label,predictions1))
#print ('Report : ')
print(classification_report(test_label, predictions1))

[[1482    3]
 [ 114 1395]]
Accuracy Score : 0.9609218436873748
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      1485
           1       1.00      0.92      0.96      1509

    accuracy                           0.96      2994
   macro avg       0.96      0.96      0.96      2994
weighted avg       0.96      0.96      0.96      2994



In [24]:
import pickle
filename = 'finalized1_text_model.sav'
pickle.dump(pipeline, open(filename, 'wb'))
 
# some time later...
 
# load the model from disk
#loaded_model = pickle.load(open(filename, 'rb'))
#result = loaded_model.score(X_test, Y_test)
#print(result)